In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sql

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


# # connect with database

In [10]:
con = sql.connect('/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db')

# # https://mystery.knightlab.com/walkthrough.html

# # Get the murder reports from SQL city

In [18]:
query_1 = '''
select * 
from crime_scene_report
where city = "SQL City"
and type = 'murder'
order by date;
'''

pd.read_sql_query(query_1,con)


,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
1,20180215,murder,REDACTED REDACTED REDACTED,SQL City
2,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


# # Description shows that there were 2 witnesses. First witness lives at the last house on Northweastern Dr. Let's retrieve the detail.


In [20]:
query_2 = '''
select * 
from person
where address_street_name = 'Northwestern Dr'
order by address_number desc;
'''
pd.read_sql_query(query_2,con)[:1]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


# # Second witness named Annabel lives somewhere on Franklin Ave

In [25]:
query_3 = '''
select * 
from person
where name like 'Annabel%' and address_street_name = 'Franklin Ave'
'''
pd.read_sql_query(query_3,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


# # Let's look at the transcript of the two witness

In [30]:
query_4 = '''
select * 
from interview
where person_id = 14887 or person_id = 16371
'''
pd.read_sql_query(query_4,con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


# # Details from get_fit_now
we got the following clues:
1. He is a gold member in Get fit now gym and his membership starts with 48Z.
2. His car number is H42W.
3. He was in the gym on 9th January.

In [31]:
query_5 = '''
select * 
from get_fit_now_check_in
where membership_id like '48Z%' and 
check_in_date = 20180109
order by check_in_date;
'''
pd.read_sql_query(query_5,con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


# # Find the details from the car number

In [34]:
query_6 = '''
select * 
from drivers_license
where plate_number like '%H42W%'
and gender = 'male'
'''
pd.read_sql_query(query_6,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


# # Find the details from the person table using the license id

In [35]:
query_7 = '''
select * 
from person
where license_id = '423327' or license_id = '664760'
'''
pd.read_sql_query(query_7,con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


# # Find the details in get_fit_now_member using the id


In [36]:
query_8 = '''
select * 
from get_fit_now_member
where person_id = '51739' or person_id = '67318'
'''
pd.read_sql_query(query_8,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


Jeremy Bowers is the murderer. Let's check this in the site mystery.knightlab.com

# ##Got the following result:

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

In [37]:
query_9 = '''
select * 
from interview
where person_id = 67318
'''
pd.read_sql_query(query_9,con)


,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


# # Let's use drivers license table to identify the person with red hair and Tesla model S.

In [39]:
query_10 = '''
select *
from drivers_license
where gender = 'female' and hair_color = 'red'
and car_make = 'Tesla' and car_model = 'Model S'
'''
pd.read_sql_query(query_10,con)


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


# # Let's find the details from person 

In [41]:
query_11 = '''
select * 
from person
where license_id = '202298' or license_id = '291182' or license_id = '918773'
'''
pd.read_sql_query(query_11,con)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


# # Let's look at facebook_event_checkin to know who attended SQL symphony concert 3 times in December 2017

In [45]:
query_12 = '''
select person_id, count(*),event_name
from facebook_event_checkin
group by person_id
having count(*) = 3 and event_name = 'SQL Symphony Concert'and date like '%201712%';
'''
pd.read_sql_query(query_12,con)

,person_id,count(*),event_name
0,24556,3,SQL Symphony Concert
1,99716,3,SQL Symphony Concert


# From query_11 and query_12, we can see it's Miranda Priestly who hired Jeremy bowers. Let's confirm this on the site mystery knightlab by inserting the name **Miranda Priestly**.
Insert into solution values(1,'Miranda Priestly');
select value from solution;

and the result is:
# Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!